## Calculate and plot clear-sky Jacobians

In [ ]:
%matplotlib inline

In [ ]:
# Set path to line catalogue
import os
if "ARTS_DATA_PATH" not in os.environ:
    os.environ["ARTS_DATA_PATH"] = "/work/um0878/users/olemke/rtcourse/arts-xml-data/"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import typhon as ty
from pyarts import xml
from jacobian_module_mpl import (calc_jacobians, plot_brightness_temperature,
                                 plot_jacobian, plot_opacity, plot_opacity_profile,
                                 argclosest)

plt.style.use(ty.plots.styles("typhon"))

In [ ]:
# Calculate Jacobians (ARTS)
jacobian_quantity = "H2O"
calc_jacobians(jacobian_quantity=jacobian_quantity)

# read in everything
freq = xml.load("results/f_grid.xml")
tau = xml.load("results/optical_thickness.xml")
bt = xml.load("results/y.xml")
jac = xml.load("results/jacobian.xml")
alt = xml.load("results/z_field.xml").ravel()
jac /= np.gradient(alt / 1000)  # normalize by layer thickness in km

In [ ]:
# select frequency
highlight_frequency = 180e9  # Hz

if highlight_frequency is None:
    fig, (ax0, ax1) = plt.subplots(ncols=2)
    plot_brightness_temperature(freq, bt, ax=ax0)
    plot_opacity(freq, tau, ax=ax1)
    freq_ind = None
else:
    fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2)
    plot_brightness_temperature(freq, bt, where=highlight_frequency, ax=ax0)
    plot_opacity(freq, tau, where=highlight_frequency, ax=ax1)
    freq_ind = argclosest(freq, highlight_frequency)
    plot_jacobian(
        alt, jac[freq_ind, :], jacobian_quantity=jacobian_quantity, ax=ax2
    )
    plot_opacity_profile(alt, tau[:, freq_ind], ax=ax3)

fig.tight_layout()
fig.savefig(f"plots/jacobians-{freq_ind}.pdf")